In [1]:
import gym
import random
import numpy as np
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from statistics import mean, median
from collections import Counter

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)


In [2]:
LR = 1e-3
env = gym.make("CartPole-v0")
env.reset()
goal_steps = 500
score_requirement = 100
initial_games = 100000

[2017-07-26 18:25:22,771] Making new env: CartPole-v0


In [3]:
def some_random_games():
    for episode in range(5):
        env.reset()
        for t in range(goal_steps):
            env.render()
            action = env.action_space.sample()
            observation, reward, done, info = env.step(action)
            if(done):
                break

In [4]:
#some_random_games()

In [5]:
def inital_population():
    train_data = []
    scores = []
    accepted_scores = []
    for _ in range(initial_games):
        score = 0
        game_memory = []
        prev_observation = []
        for _ in range(goal_steps):
            action = random.randrange(0,2)
            #action = env.action_space.sample()
            observation, reward, done, info = env.step(action)
            if len(prev_observation) > 0:
                game_memory.append([prev_observation,action])
                
            prev_observation = observation
            score += reward
            if done:
                break
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                if data[1] == 1:
                    output = [0,1]
                elif data[1] == 0:
                    output = [1,0]
                train_data.append([data[0], output])
        env.reset()
        scores.append(score)
    train_data_save = np.array(train_data)
    np.save('saved.npy', train_data_save)
    
    print('Average accepted score: ', mean(accepted_scores))
    print('Median accepted score: ', median(accepted_scores))
    print(Counter(accepted_scores))
    
    return train_data
    

In [6]:
#inital_population()

In [7]:
def neural_network_model(input_size):
    network = input_data(shape=[None, input_size, 1], name='input')
    
    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 1024, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 2, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')
    
    model = tflearn.DNN(network, tensorboard_dir='log')
    
    return model

In [8]:
def train_model(train_data, model=False):
    x = np.array([i[0] for i in train_data]).reshape(-1,len(train_data[0][0]), 1)
    y = [i[1] for i in train_data]
    
    if not model:
        model = neural_network_model(input_size= len(x[0]))
        
    model.fit({'input':x}, {'targets':y}, n_epoch=3, snapshot_step=100, show_metric=True,run_id='openaistuff')
    
    return model

In [9]:
def play_game(visible=True):
    scores = []
    choices = []

    for each_game in range(100):
        score = 0
        game_memory = []
        prev_obs = []
        env.reset()
        for _ in range(goal_steps):
            if visible:
                env.render()
            if len(prev_obs) == 0:
                action = random.randrange(0,2)
            else:
                action = np.argmax(model.predict(prev_obs.reshape(-1, len(prev_obs), 1))[0])
            choices.append(action)

            new_obs, reward, done, info = env.step(action)
            prev_obs = new_obs
            game_memory.append([new_obs, action])
            score += reward
            if done:
                break
        scores.append(score)
    average_score = sum(scores)/len(scores)
    if visible:
        print('Scores: ',scores)
        print('Average Score ', average_score)
        print('Choice 0: {}, Choice 1: {}'.format(choices.count(0)/len(choices), choices.count(1)/len(choices)))
    return average_score


In [ ]:
#training_data = inital_population()
#model = train_model(training_data)
#average_score = play_game(False)
#while average_score < 300:
#    training_data = inital_population()
#    model = train_model(training_data, model)
#    average_score = play_game(False)
#play_game()

In [ ]:
training_data = inital_population()
model = train_model(training_data)
play_game()

Training Step: 296  | total loss: 0.64030 | time: 5.167s
| Adam | epoch: 003 | loss: 0.64030 - acc: 0.6414 -- iter: 6272/6284
Training Step: 297  | total loss: 0.64424 | time: 5.234s
| Adam | epoch: 003 | loss: 0.64424 - acc: 0.6335 -- iter: 6284/6284
--
